# Dsuite

### Use Caro notes and new subsetted vcf with hybrids


### INSTALLED in /data/gpfs/assoc/denovo/src/Dsuite
https://github.com/millanek/Dsuite/blob/master/README.md

### NOTE!!!! Make sure you module load gcc/9.2.0 or some verison first

In [96]:
!module load gcc/9.2.0

In [12]:
import sys
import ipyparallel as ipp
import os
from os import environ
import gzip
import warnings
import pandas as pd
import numpy as np
import scipy as sp
import glob
import re
import random

In [7]:
Dsuite = '/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite'

In [8]:
root = '/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo'

In [9]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo


In [10]:
!mkdir Dsuite

mkdir: cannot create directory ‘Dsuite’: File exists


In [13]:
D_dir = os.path.join(root,'Dsuite')

In [14]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


In [15]:
!cp ../subset_vcf/ERNA_sub2.recode.vcf.gz . 

In [16]:
!cp ../subset_vcf/Pop_ID_sub2.csv . 

In [17]:
!cp ../iqtree/ERNA_sub2.recode.min4.phy.varsites.phy.contree . 

### Make SETS.txt

In [23]:
Pop_ID_sub2 = pd.read_csv('Pop_ID_sub2.csv')
Pop_ID_sub2['Pop_lin'] = Pop_ID_sub2['Pop'] + '_' + Pop_ID_sub2['Lin']
Pop_ID_sub2

,Pop,ID,All,Name,Lat,Long,State,Elevation,Variety,Ssp,Lin,Pop_lin
0,AR,2,EN_AR_2,Arenaria1,37.17384,-112.61817,UT,1870,arenaria,consimilis,green,AR_green
1,AR,3,EN_AR_3,Arenaria1,37.17384,-112.61817,UT,1870,arenaria,consimilis,green,AR_green
2,AR,4,EN_AR_4,Arenaria1,37.17384,-112.61817,UT,1870,arenaria,consimilis,green,AR_green
3,AR,5,EN_AR_5,Arenaria1,37.17384,-112.61817,UT,1870,arenaria,consimilis,green,AR_green
4,AR,6,EN_AR_6,Arenaria1,37.17384,-112.61817,UT,1870,arenaria,consimilis,green,AR_green
...,...,...,...,...,...,...,...,...,...,...,...,...
226,YL,5,EN_YL_5,10-3 yellow,43.53928,-119.43722,OR,1307,speciosa,nauseosa,grey,YL_grey
227,YL,6,EN_YL_6,10-3 yellow,43.53928,-119.43722,OR,1307,speciosa,nauseosa,grey,YL_grey
228,ED,1,EN_ED_1,E.discoidea,45.64621,-111.52273,NV,1334,E.discoidea,E.discoidea,outgroup,ED_outgroup
229,ED,2,EN_ED_2,E.discoidea,45.64621,-111.52273,NV,1334,E.discoidea,E.discoidea,outgroup,ED_outgroup


In [43]:
sets_sub2 = pd.DataFrame(data={'indv': Pop_ID_sub2['All'],
                               'pop': Pop_ID_sub2['All']})

In [45]:
sets_sub2['pop'][sets_sub2['pop'].isin(['EN_ED_1','EN_ED_2','EN_ED_3'])] = 'Outgroup'

In [46]:
sets_sub2.to_csv('sets_sub2.txt',header=None,index=False,sep='\t')

In [48]:
!tail sets_sub2.txt

EN_UT_4	EN_UT_4
EN_UT_5	EN_UT_5
EN_YL_1	EN_YL_1
EN_YL_3	EN_YL_3
EN_YL_4	EN_YL_4
EN_YL_5	EN_YL_5
EN_YL_6	EN_YL_6
EN_ED_1	Outgroup
EN_ED_2	Outgroup
EN_ED_3	Outgroup


### Convert tree from iqtree to correct Newick format 

remove nodelabels / bootstraps 

In [38]:
%load_ext rpy2.ipython

In [91]:
%%R
setwd('/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite')
library(ape)


tree <- read.tree("ERNA_sub2.recode.min4.phy.varsites.phy.contree")
tree <- root(tree,outgroup = c('EN_ED_1','EN_ED_2','EN_ED_3'), resolve.root = TRUE)
tree$node.label <- NULL
tree$edge.length <- NULL
write.tree(tree, file = "ERNA_sub2.nwk.tree")

## Run Dtrios
./Dsuite Dtrios erna_sub.vcf SETS.txt -t tree_sub_all.tree

In [32]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


#### NOTE: ran once but invariant sites. rerun with new .phy file

In [49]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'Dtrios'
time = '8-00:00:00' #time limit 1 day
cpus = 16
mem_cpu = 9000
email = 'tfaske@nevada.unr.edu'

### for Dtrios
Dsuite = '/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite'
vcf_file = os.path.join(D_dir,'ERNA_sub2.recode.vcf.gz')
sets_file = os.path.join(D_dir,'sets_sub2.txt')
tree_file = os.path.join(D_dir,'ERNA_sub2.nwk.tree')

In [50]:
def write_Dtrios_sh(account,partition,time,cpus,mem_cpu,email,D_dir,Dsuite,vcf_file,sets_file,tree_file):
    with open("run_Dtrios_sub2.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name Dtrios_sub2
#SBATCH --output output_Dtrios_sub2.txt
#SBATCH --mail-type=END
#SBATCH --mail-user=%s

cd %s
    
%s Dtrios %s %s -t %s \n\n""" % (account,partition,time,int(cpus),int(mem_cpu),email,
                                 D_dir,Dsuite,vcf_file,sets_file,tree_file))
        

In [51]:
write_Dtrios_sh(account,partition,time,cpus,mem_cpu,email,D_dir,Dsuite,vcf_file,sets_file,tree_file)

### Run Dtrios.sh
    cd /data/gpfs/home/tfaske/d/rabbit/full/phylo/Dsuite
    source activate py311
    module load gcc/9.2.0
    sbatch run_iqtree.sh

# Run Fbranch (do not need slurm)
./Dsuite Fbranch -Z --Pb-matrix tree_sub_all.tree SETS_tree.txt > fbranch_collapsed_p.txt

In [32]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


In [98]:
### for Dtrios
Dsuite = '/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite'
sets_tree_file = os.path.join(D_dir,'sets_sub2_tree.txt')
tree_file = os.path.join(D_dir,'ERNA_sub2.nwk.tree')
out_file = os.path.join(D_dir,'fbranch_sub2.txt')

### command 
fb_cmd = '%s Fbranch --Zb-matrix --Pb-matrix %s %s > %s' % (Dsuite,tree_file,sets_tree_file,out_file)

In [99]:
fb_cmd

'/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite Fbranch --Zb-matrix --Pb-matrix /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/ERNA_sub2.nwk.tree /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/sets_sub2_tree.txt > /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/fbranch_sub2.txt'

#### whatever, run in command line

# Run Dsuite on onetip! 

In [52]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


In [53]:
!cp ../subset_vcf/ERNA_onetip.recode.vcf.gz . 

In [54]:
!cp ../subset_vcf/Pop_ID_onetip.csv . 

In [55]:
!cp ../iqtree/ERNA_onetip.recode.min4.phy.varsites.phy.contree . 

### Make SETS.txt

In [57]:
Pop_ID_onetip = pd.read_csv('Pop_ID_onetip.csv')
Pop_ID_onetip.head()

,Pop,ID,All,Name,Lat,Long,State,Elevation,Variety,Ssp,Lin,Pop_lin
0,AR,4,EN_AR_4,Arenaria1,37.173840,-112.618170,UT,1870,arenaria,consimilis,green,AR_green
1,AS,29,EN_AS_29,Austin Summit,39.473104,-117.049449,NV,2408,hololueca,nauseosa,grey,AS_grey
2,BC,10,EN_BC_10,beaver canyon,38.279820,-112.568070,UT,1887,salicifolia,nauseosa,green,BC_green
3,BC,5,EN_BC_5,beaver canyon,38.279820,-112.568070,UT,1887,salicifolia,nauseosa,hybrid,BC_hybrid
4,BH,5,EN_BH_5,Baker Hwy,45.089070,-117.848370,OR,1007,oreophila,consimilis,green,BH_green


In [59]:
sets_onetip = pd.DataFrame(data={'indv': Pop_ID_onetip['All'],
                                 'pop': Pop_ID_onetip['All']})

In [60]:
sets_onetip['pop'][sets_onetip['pop']=='EN_ED_1'] = 'Outgroup'

In [61]:
sets_onetip.to_csv('sets_onetip.txt',header=None,index=False,sep='\t')

In [62]:
!tail sets_onetip.txt

EN_PW_2	EN_PW_2
EN_RH_5	EN_RH_5
EN_RS_2	EN_RS_2
EN_ST_9	EN_ST_9
EN_SS_2	EN_SS_2
EN_SS_14	EN_SS_14
EN_TO_5	EN_TO_5
EN_UT_3	EN_UT_3
EN_YL_4	EN_YL_4
EN_ED_1	Outgroup


### Convert tree from iqtree to correct Newick format 

remove nodelabels / bootstraps and edge lengths and ROOT

In [63]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [130]:
%%R
setwd('/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite')
library(ape)


tree <- read.tree("ERNA_onetip.recode.min4.phy.varsites.phy.contree")
tree <- root(tree,outgroup = c('EN_ED_1'), resolve.root = TRUE)
tree$node.label <- NULL
tree$edge.length <- NULL
#str(tree)
write.tree(tree, file = "ERNA_onetip.nwk.tree")

## Run Dtrios
./Dsuite Dtrios erna_sub.vcf SETS.txt -t tree_sub_all.tree

In [65]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


#### NOTE: ran once but invariant sites. rerun with new .phy file

In [66]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'Dtrios'
time = '8-00:00:00' #time limit 1 day
cpus = 16
mem_cpu = 9000
email = 'tfaske@nevada.unr.edu'

### for Dtrios
Dsuite = '/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite'
vcf_file = os.path.join(D_dir,'ERNA_onetip.recode.vcf.gz')
sets_file = os.path.join(D_dir,'sets_onetip.txt')
tree_file = os.path.join(D_dir,'ERNA_onetip.nwk.tree')

In [67]:
def write_Dtrios_sh(account,partition,time,cpus,mem_cpu,email,D_dir,Dsuite,vcf_file,sets_file,tree_file):
    with open("run_Dtrios_onetip.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name Dtrios_onetip
#SBATCH --output output_Dtrios_onetip.txt
#SBATCH --mail-type=END
#SBATCH --mail-user=%s

cd %s
    
%s Dtrios %s %s -t %s \n\n""" % (account,partition,time,int(cpus),int(mem_cpu),email,
                                 D_dir,Dsuite,vcf_file,sets_file,tree_file))
        

In [68]:
write_Dtrios_sh(account,partition,time,cpus,mem_cpu,email,D_dir,Dsuite,vcf_file,sets_file,tree_file)

### Run Dtrios.sh
    cd /data/gpfs/home/tfaske/d/rabbit/full/phylo/Dsuite
    source activate py311
    module load gcc/9.2.0
    sbatch run_iqtree.sh

# Run Fbranch (do not need slurm)
./Dsuite Fbranch -Z --Pb-matrix tree_sub_all.tree SETS_tree.txt > fbranch_collapsed_p.txt

#### Note, sets_tree from Dsuite

In [101]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


In [109]:
### for Fbranch 
Dsuite = '/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite'
sets_tree_file = os.path.join(D_dir,'sets_onetip_tree.txt')
tree_file = os.path.join(D_dir,'ERNA_onetip.nwk.tree')
out_p_file = 'p_onetip.txt'
out_z_file = 'z_onetip.txt'

### command 
fb_p_cmd = '%s Fbranch --Pb-matrix %s %s > fbranch_%s' % (Dsuite,tree_file,sets_tree_file,out_p_file)
fb_z_cmd = '%s Fbranch --Zb-matrix %s %s > fbranch_%s' % (Dsuite,tree_file,sets_tree_file,out_z_file)

In [110]:
fb_p_cmd

'/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite Fbranch --Pb-matrix /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/ERNA_onetip.nwk.tree /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/sets_onetip_tree.txt > fbranch_p_onetip.txt'

In [111]:
fb_z_cmd

'/data/gpfs/assoc/denovo/src/Dsuite/Build/./Dsuite Fbranch --Zb-matrix /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/ERNA_onetip.nwk.tree /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/sets_onetip_tree.txt > fbranch_z_onetip.txt'

#### whatever, run in command line

#### Select just z for plotting, only keep those above 0.05 or 5%

In [115]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


In [118]:
!head -n 91 fbranch_z_onetip.txt > fbranch_plotting_onetip.txt

In [122]:
%%R
zscores <- read.csv("fbranch_plotting_onetip.txt", sep = "\t")
#head(zscores)
zscores[zscores < 0.05] <- 0
write.table(zscores, "fbranch_onetip_5.txt", sep='\t',row.names=F, quote=F)

## Run dtools to get figure
./dtools.py f_5_percent_plotting_trimmed.txt tree_sub_trimmed.tree --outgroup EN_ED --ladderize

In [123]:
cd $D_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite


In [128]:
dtools = '/data/gpfs/home/tfaske/ad/src/Dsuite/utils/./dtools.py'
z_5_file = os.path.join(D_dir,'fbranch_onetip_5.txt')
tree_file = os.path.join(D_dir,'ERNA_onetip.nwk.tree')
outgroup = 'EN_ED_1'
out_plot = 'fb_onetip_plot'

### command 
dtools_cmd = 'python3 %s  %s %s --outgroup %s --ladderize -n %s' % (dtools,z_5_file,tree_file,outgroup,out_plot)

In [129]:
dtools_cmd

'python3 /data/gpfs/home/tfaske/ad/src/Dsuite/utils/./dtools.py  /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/fbranch_onetip_5.txt /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/ERNA_onetip.nwk.tree --outgroup EN_ED_1 --ladderize -n fb_onetip_plot'

### DOES NOT WORK WITH PY311, ust run with py3.6
#### whatever run in command line

In [ ]:
/data/gpfs/home/tfaske/ad/src/Dsuite/utils/./dtools.py  /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/fbranch_onetip_5.txt /data/gpfs/assoc/denovo/tfaske/rabbit/full/REDO/phylo/Dsuite/ERNA_onetip.nwk.tree --outgroup EN_ED_1 --ladderize -n fb_onetip_plot